In [11]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string

In [12]:
spam_df = pd.read_csv('spam.csv')

In [13]:
spam_df

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [14]:
spam_df.columns

Index(['Category', 'Message'], dtype='object')

In [15]:
spam_df['Category']= spam_df['Category'].map({'spam': 1, 'ham': 0})

In [16]:
spam_df

,Category,Message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...
5568,0,Will ü b going to esplanade fr home?
5569,0,"Pity, * was in mood for that. So...any other s..."
5570,0,The guy did some bitching but I acted like i'd...


In [17]:
spam_df.shape

(5572, 2)

In [18]:
spam_df['Category'].value_counts()

Category
0    4825
1     747
Name: count, dtype: int64

In [19]:
# dropping all the duplicate

In [20]:
spam_df.drop_duplicates(inplace=True)

In [21]:
spam_df.shape

(5157, 2)

In [22]:
#ceheck if there any null,NaN, Na value

In [23]:
spam_df.isnull().sum()

Category    0
Message     0
dtype: int64

In [24]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [25]:
# removing punctuation
# remove stopwords
# return a list of clean text

def process_text(text):
    no_punc = [char for char in text if char not in string.punctuation]
    no_punc = ''.join(no_punc)
    
    clean_words = [ word for word in no_punc.split() if word.lower() not in stopwords.words('english')]
    
    return clean_words

In [26]:
# show a list of tokens also known as lemmas

spam_df['Message'].head().apply(process_text)

0    [Go, jurong, point, crazy, Available, bugis, n...
1                       [Ok, lar, Joking, wif, u, oni]
2    [Free, entry, 2, wkly, comp, win, FA, Cup, fin...
3        [U, dun, say, early, hor, U, c, already, say]
4    [Nah, dont, think, goes, usf, lives, around, t...
Name: Message, dtype: object

In [27]:
# convert a collection of txt of a matrix of tekans
from sklearn.feature_extraction.text import CountVectorizer

message_bagofwords = CountVectorizer(analyzer=process_text).fit_transform(spam_df['Message'])

In [28]:
message_bagofwords.shape

(5157, 11422)

In [29]:
# split the data into 80% and 20% train test acc.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train,y_test = train_test_split(message_bagofwords,spam_df["Category"],test_size=0.20,random_state=0)

In [30]:
X_train

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 36747 stored elements and shape (4125, 11422)>

In [31]:
X_test.shape

(1032, 11422)

In [32]:
X_train.shape

(4125, 11422)

In [33]:
type(X_test)

scipy.sparse._csr.csr_matrix

In [34]:
y_train

3734    0
3720    1
681     0
4165    0
3057    0
       ..
5320    0
3462    0
1711    0
2735    0
2877    0
Name: Category, Length: 4125, dtype: int64

In [46]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer

#you can only imoport if its doesn't run the code.

In [41]:
# create and train Naive Bayes Classifier
from sklearn.naive_bayes import MultinomialNB
MNBClassifier = MultinomialNB().fit(X_train,y_train)

In [42]:
# calculating accuracy on training data
from sklearn.metrics import classification_report, confusion_matrix

y_train_pred  = MNBClassifier.predict(X_train)
print("Accuracy score on training data :")
print(accuracy_score(y_train_pred,y_train))
print("Classification report on training data :")
print(classification_report(y_train_pred,y_train))
print()
print("Confusion matrix on training data :")
print(confusion_matrix(y_train_pred,y_train))

Accuracy score on training data :
0.9949090909090909
Classification report on training data :
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3624
           1       0.97      0.98      0.98       501

    accuracy                           0.99      4125
   macro avg       0.99      0.99      0.99      4125
weighted avg       0.99      0.99      0.99      4125


Confusion matrix on training data :
[[3611   13]
 [   8  493]]


In [43]:
# calculating accuracy on testing data
y_pred = MNBClassifier.predict(X_test)
print(accuracy_score(y_pred,y_test))

0.9612403100775194


In [38]:
print(y_pred)
print()

[0 0 0 ... 0 0 0]



In [44]:
print("Classification report on testing data : \n",classification_report(y_pred,y_test))
print()
print("Confusion matrix on testing data : \n", confusion_matrix(y_pred,y_test))

Classification report on testing data : 
               precision    recall  f1-score   support

           0       0.97      0.99      0.98       877
           1       0.93      0.81      0.86       155

    accuracy                           0.96      1032
   macro avg       0.95      0.90      0.92      1032
weighted avg       0.96      0.96      0.96      1032


Confusion matrix on testing data : 
 [[867  10]
 [ 30 125]]
